# Autoencoder

## Build Autoencoder

In [85]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

In [86]:
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.0-arm64-arm-64bit
Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
Pandas 1.5.3
Scikit-Learn 1.2.2
GPU is available


In [87]:
# Load the data
import numpy as np
data = np.load('/Users/xinyu/Downloads/noisy_training.npy')

In [88]:
print(data)

[[[[0.94509291 0.96072069 0.9134108 ]
   [0.79577509 1.         0.85024139]
   [0.97882846 0.97563394 0.85039027]
   ...
   [0.89050754 1.         0.87407183]
   [0.98919572 1.         0.95115111]
   [0.92072082 0.97342284 0.91186438]]

  [[0.97284576 0.95586043 0.90373644]
   [0.9390784  1.         0.89121266]
   [0.9491018  0.97018584 0.78276212]
   ...
   [0.9506112  1.         0.89834631]
   [0.94126845 1.         0.97375394]
   [0.84459862 1.         0.95365493]]

  [[0.93913914 0.99784681 0.92527942]
   [0.88305785 0.99252652 0.93041087]
   [0.91000618 0.89614722 0.93729248]
   ...
   [0.92749181 0.9124723  0.97213496]
   [0.99101137 0.94199754 0.89991337]
   [0.95389765 1.         0.94087955]]

  ...

  [[0.76710226 1.         0.93201475]
   [0.9308711  0.85684552 0.80611415]
   [0.95110664 1.         0.84233801]
   ...
   [0.86131573 0.97872846 0.93909401]
   [0.95706232 0.99481219 0.88331919]
   [0.87159059 1.         0.95504079]]

  [[0.89674333 0.90826828 0.87364126]
   [1. 

In [89]:
# Find the range of pixel values in the image data
max_value = np.max(data)
min_value = np.min(data)
print("Maximum pixel value:", max_value)
print("Minimum pixel value:", min_value)

Maximum pixel value: 1.0
Minimum pixel value: 0.0


In [91]:
# Calculate the mean and standard deviation of the pixel values
pixel_mean = np.mean(data)
pixel_std = np.std(data)

print("Pixel mean:", pixel_mean)
print("Pixel standard deviation:", pixel_std)

Pixel mean: 0.5106262173898898
Pixel standard deviation: 0.25190882925364866


In [92]:
# Get the shape of the data
print(data.shape)

(95460, 32, 32, 3)


In [102]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense

In [103]:
# Encoder
input_shape = (32, 32, 3)
input_img = Input(shape=input_shape)
x = Conv2D(48, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(96, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(32, (1, 1), activation='relu', padding='same')(x)


In [104]:
# LATENT SPACE
latentSize = (4,4,32)

In [105]:
# Decoder
direct_input = Input(shape=latentSize)
x = Conv2D(192, (1, 1), activation='relu', padding='same')(direct_input)
x = UpSampling2D((2, 2))(x)
x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(96, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(48, (3, 3), activation='relu', padding='same')(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [106]:
# Define the model
encoder = Model(input_img, encoded)
decoder = Model(direct_input, decoded)
autoencoder = Model(input_img, decoder(encoded))

autoencoder.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_58 (Conv2D)          (None, 32, 32, 48)        1344      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 16, 16, 48)       0         
 g2D)                                                            
                                                                 
 conv2d_59 (Conv2D)          (None, 16, 16, 96)        41568     
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 8, 8, 96)         0         
 g2D)                                                            
                                                                 
 conv2d_60 (Conv2D)          (None, 8, 8, 192)         166

In [107]:
# Compile the model
autoencoder.compile(optimizer='Adam', loss='binary_crossentropy')

## Train Autoencoder

In [108]:
# Train the model
autoencoder.fit(data, data, epochs=10)

Epoch 1/10
2984/2984 [==============================] - 192s 62ms/step - loss: 0.5720
Epoch 2/10
2984/2984 [==============================] - 210s 70ms/step - loss: 0.5626
Epoch 3/10
2984/2984 [==============================] - 212s 71ms/step - loss: 0.5614
Epoch 4/10
2984/2984 [==============================] - 209s 70ms/step - loss: 0.5608
Epoch 5/10
2984/2984 [==============================] - 211s 71ms/step - loss: 0.5604
Epoch 6/10
2984/2984 [==============================] - 214s 72ms/step - loss: 0.5602
Epoch 7/10
2984/2984 [==============================] - 210s 71ms/step - loss: 0.5599
Epoch 8/10
2984/2984 [==============================] - 209s 70ms/step - loss: 0.5598
Epoch 9/10
2984/2984 [==============================] - 211s 71ms/step - loss: 0.5597
Epoch 10/10
2984/2984 [==============================] - 208s 70ms/step - loss: 0.5596


## View Reconstruction

In [109]:
import matplotlib.pyplot as plt